# Movie Recommendation System


Import libraries

In [ ]:
import pandas as pd
import numpy as np
import tqdm as tqdm
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
import xgboost as xg
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error

Load the Train data

In [ ]:
Movie_data = pd.read_table("train.dat",sep=' ')
Movie_data.head()

,userID,movieID,rating
0,75,3,1.0
1,75,32,4.5
2,75,110,4.0
3,75,163,4.0
4,75,165,4.5


In [ ]:
Movie_data.corr()

,userID,movieID,rating
userID,1.000000,0.002363,-0.009343
movieID,0.002363,1.000000,0.012484
rating,-0.009343,0.012484,1.000000


Split the ratings and UserId, MovieID columns

In [ ]:
Movie_rating = Movie_data['rating']
Movie_data1 = Movie_data.drop(['rating'], axis=1)


Non Negative Matrix Factorization on Train data

In [ ]:
user_movie_matrix = pd.pivot_table(Movie_data,values='rating',index='userID',columns='movieID')
user_movie_matrix

movieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,64245,64249,64273,64278,64280,64285,64288,64338,64368,64418,64427,64497,64499,64501,64508,64519,64575,64614,64620,64622,64701,64716,64750,64839,64900,64930,64942,64957,64969,64983,64986,64990,64993,64997,65011,65037,65088,65126,65130,65133
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
75,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71497,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71509,4.0,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from sklearn.decomposition import NMF
model = NMF(n_components=3, init='random', random_state=0)
W = model.fit_transform(Movie_data)
H = model.components_

In [ ]:
W

array([[0.00000000e+00, 1.25142027e-02, 8.29670525e-01],
       [6.58299178e-02, 1.33344690e-01, 7.27365110e-01],
       [4.08779993e-01, 4.57978350e-01, 2.20812446e-01],
       ...,
       [5.53086949e+02, 1.93110628e+02, 4.09555108e+00],
       [5.46544282e+02, 2.53597211e+02, 7.97478475e+00],
       [5.45682132e+02, 2.57661986e+02, 8.82479239e+00]])

Split the training data into train and test.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Movie_data1, Movie_rating, random_state= 30, test_size=0.3)

Implementing XGBoost Regressor

In [ ]:
import xgboost as xg
grad = xg.XGBRegressor(n_jobs=20,random_state=0,n_estimators=2000)
clf1 = grad.fit(X_train, y_train)
predicted1= clf1.predict(X_test)
#print("RMSE value", mean_squared_error(y_test, predicted1, squared=False))
print(predicted1)

[19:34:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[3.3136418 2.9379528 3.385287  ... 3.210211  3.8772554 3.2172463]


Implementing KNN Regressor

In [ ]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import neighbors
knn = neighbors.KNeighborsRegressor(n_neighbors=300, metric='cosine')
clf1 = knn.fit(X_train, y_train)
predicted1= clf1.predict(X_test)
print("RMSE value", mean_squared_error(y_test, predicted1, squared=False))


RMSE value 0.9993136064545871


Stochastic Gradient Descent

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

reg = make_pipeline(StandardScaler(), SGDRegressor(max_iter=1000, tol=1e-3))
#reg = SGDRegressor(max_iter=1000, tol=1e-3)
reg.fit(X_train, y_train)
predicted1= reg.predict(X_test)
print("RMSE value", mean_squared_error(y_test, predicted1, squared=False))
print(predicted1)

RMSE value 0.9991956503865251
[3.43876689 3.42052129 3.42672369 ... 3.42589537 3.41560534 3.45216317]


Naive Bayes - Bayesian Ridge Regressor

In [ ]:
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error
bayesian = BayesianRidge()
clf1 = bayesian.fit(X_train, y_train)
predicted1= clf1.predict(X_test)
print("RMSE value", mean_squared_error(y_test, predicted1, squared=False))

RMSE value 0.9989588005986523


Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regr_2 = DecisionTreeRegressor(max_depth=5)
clf1 = regr_2.fit(X_train, y_train)
predicted1= clf1.predict(X_test)
print("RMSE value", mean_squared_error(y_test, predicted1, squared=False))

RMSE value 0.9836083216132724


AdaBoost Regressor

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
regr = AdaBoostRegressor(random_state=0, n_estimators=100)
clf1 = regr.fit(X_train, y_train)
predicted1= clf1.predict(X_test)
print("RMSE value", mean_squared_error(y_test, predicted1, squared=False))

RMSE value 1.0153513714301599


Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=2, random_state=0)
clf1 = regr.fit(X_train, y_train)
predicted1= clf1.predict(X_test)
print("RMSE value", mean_squared_error(y_test, predicted1, squared=False))

RMSE value 0.9890707407369254


Matrix Factorization on Test Data

In [ ]:
from sklearn.decomposition import NMF
Movie_test_data = pd.read_table("test.dat",sep=' ')
model = NMF(n_components=3, init='random', random_state=0)
W = model.fit_transform(Movie_test_data)
H = model.components_
W

array([[2.11752610e-01, 2.40371443e+00, 8.83985092e-02],
       [1.39460234e-01, 5.62154415e+00, 2.02679019e-04],
       [0.00000000e+00, 1.19896628e+01, 0.00000000e+00],
       ...,
       [3.13976740e+02, 1.55690787e+01, 9.63302610e+01],
       [3.51397666e+02, 2.60653142e+01, 6.48581177e+01],
       [4.23785136e+02, 1.31927631e+02, 4.12249385e-02]])

## **TESTING DATA - Using XGBRegressor**

In [ ]:
Movie_test_data = pd.read_csv("test.dat",sep=' ')
grad = xg.XGBRegressor(n_jobs=20,random_state=0,n_estimators=2000,learning_rate=1.0)
clf1 = grad.fit(X_train, y_train)
y_pred = clf1.predict(Movie_test_data)


[02:16:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


## **Generating Final Output File**

In [ ]:
with open('test_xg_final.txt', 'w') as f:
    for i in y_pred:
        f.write(str(i)+'\n')

## Working on Genre as an additional feature

In [ ]:
!pip install tqdm
import tqdm

In [ ]:
#taking unique values
train = pd.read_table('train.dat', skip_blank_lines=False,delim_whitespace=True)
test = pd.read_table('test.dat', skip_blank_lines=False,delim_whitespace=True)
genre = pd.read_table('movie_genres.dat',skip_blank_lines=False,delim_whitespace=True)

users = train['userID'].unique()
users = dict(zip(users, np.arange(users.size)))
movies = train['movieID'].unique()
movies = dict(zip(movies, np.arange(movies.size)))
genres = genre['genre'].unique()


In [ ]:
showiters = tqdm.tqdm(total=len(train))

g=[]
for index,row in train.iterrows():
    genre_list = ""
    x = genre.loc[genre['movieID']==row.movieID]
    for i,r in x.iterrows():
        genre_list += r.genre + ","
    g.append(genre_list)
    showiters.update(1)
showiters.close()

Streaming output truncated to the last 5000 lines.
 96%|█████████▌| 614855/641699 [11:24<00:32, 819.72it/s]

In [58]:
rating = train["rating"]*10
del train["rating"]

In [59]:
train['genres'] = g
for genre in list(genres) :
    train[genre] = train.apply(lambda x:int(genre in x.genres), axis = 1)

In [60]:
del train['genres']

In [61]:
clf = xg.XGBRegressor(n_jobs=150, random_state=150, n_estimators=1000)
clf.fit(train,rating)

[03:41:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=150, nthread=None, objective='reg:linear', random_state=150,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [62]:
gen2 = pd.read_table('movie_genres.dat',skip_blank_lines=False,delim_whitespace=True)

In [63]:
showiters = tqdm.tqdm(total=len(test))

g2=[]
for index,row in test.iterrows():
    genre_list = ""
    x = gen2.loc[gen2['movieID']==row.movieID]
    for i,r in x.iterrows():
        genre_list += r.genre + ","
    g2.append(genre_list)
    showiters.update(1)
showiters.close()

100%|██████████| 71299/71299 [01:16<00:00, 936.42it/s]


In [64]:
test['genres'] = g2
for genre in list(genres) :
    test[genre] = test.apply(lambda x:int(genre in x.genres), axis = 1)

In [65]:
del test['genres']

In [66]:
preds = clf.predict(test)

In [67]:
f=open("./Output4.txt","w")
for a in preds:
    f.write(str(float(a/10)))
    f.write("\n")
f.close()

# Working on data files from Additional Files folder.

Merge Actors data set

In [ ]:
data = pd.read_table('movie_actors.dat',skip_blank_lines=False,encoding='latin-1')
data.head()

,movieID,actorID,actorName,ranking
0,1,annie_potts,Annie Potts,10
1,1,bill_farmer,Bill Farmer,20
2,1,don_rickles,Don Rickles,3
3,1,erik_von_detten,Erik von Detten,13
4,1,greg-berg,Greg Berg,17


In [ ]:
new_data1 = pd.merge(data,Movie_data)

In [ ]:
new_data1

,movieID,actorID,actorName,ranking,userID,rating
0,1,annie_potts,Annie Potts,10,170,3.0
1,1,annie_potts,Annie Potts,10,493,3.5
2,1,annie_potts,Annie Potts,10,533,5.0
3,1,annie_potts,Annie Potts,10,545,4.0
4,1,annie_potts,Annie Potts,10,622,4.0
...,...,...,...,...,...,...
17960652,65133,stephen_fry,Stephen Fry,8,33384,3.0
17960653,65133,tim_mcinnerny,Tim McInnerny,9,24495,4.0
17960654,65133,tim_mcinnerny,Tim McInnerny,9,33384,3.0
17960655,65133,tony_robinson,Tony Robinson,10,24495,4.0


In [ ]:
from sklearn import preprocessing
label = preprocessing.LabelEncoder()
new_data1['actorName'] = label.fit_transform(new_data1['actorName'].astype('str'))
new_data1['actorID'] = label.fit_transform(new_data1['actorID'].astype('str'))

In [ ]:
new_data1

,movieID,actorID,actorName,ranking,userID,rating
0,1,8020,5072,10,170,3.0
1,1,8020,5072,10,493,3.5
2,1,8020,5072,10,533,5.0
3,1,8020,5072,10,545,4.0
4,1,8020,5072,10,622,4.0
...,...,...,...,...,...,...
17960652,65133,83765,83270,8,33384,3.0
17960653,65133,87539,87226,9,24495,4.0
17960654,65133,87539,87226,9,33384,3.0
17960655,65133,88941,88676,10,24495,4.0


Merge Movie Tags

In [ ]:
movie_tag = pd.read_table('user_taggedmovies.dat',sep=' ')
movie_tag

,userID,movieID,tagID
0,75,353,5290
1,78,4223,5264
2,127,1343,1544
3,127,1343,12330
4,127,2080,1451
...,...,...,...
47952,71534,7937,306
47953,71534,8848,331
47954,71534,8848,427
47955,71534,25833,7671


In [ ]:
new_data = pd.merge(movie_tag,Movie_data)
new_data

,userID,movieID,tagID,rating
0,75,353,5290,3.5
1,78,4223,5264,5.0
2,175,2819,11407,5.0
3,175,51540,79,4.0
4,190,741,712,4.0
...,...,...,...,...
27681,71534,3307,7670,5.0
27682,71534,3448,177,5.0
27683,71534,4357,4804,4.5
27684,71534,7084,61,4.5


Merge Movie Genres

In [ ]:
movie_genres = pd.read_table('movie_genres.dat',sep='\t')
movie_genres.head()

,movieID,genre
0,1,Adventure
1,1,Animation
2,1,Children
3,1,Comedy
4,1,Fantasy


In [68]:
new_data1=pd.merge(new_data1,movie_genres)

In [69]:
new_data1

,movieID,actorID,actorName,ranking,userID,rating,genre
0,1,8020,5072,10,170,3.0,Adventure
1,1,8020,5072,10,170,3.0,Animation
2,1,8020,5072,10,170,3.0,Children
3,1,8020,5072,10,170,3.0,Comedy
4,1,8020,5072,10,170,3.0,Fantasy
...,...,...,...,...,...,...,...
46785082,65133,83765,83270,8,33384,3.0,Comedy
46785083,65133,87539,87226,9,24495,4.0,Comedy
46785084,65133,87539,87226,9,33384,3.0,Comedy
46785085,65133,88941,88676,10,24495,4.0,Comedy
